In [1]:
import re, json, pandas as pd, numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error


In [4]:
ROOT_DIR   = Path.cwd().parent          #  .../root/
DATA_DIR   = ROOT_DIR / "data" / "test data"   #  .../root/data/test_data/

USERS_CSV  = DATA_DIR / "ten_users_gpt-4o.csv"
MEALS_CSV  = DATA_DIR / "test_meal_logs.csv"

print("Users CSV :", USERS_CSV)
print("Meals CSV :", MEALS_CSV)

users_df = pd.read_csv(USERS_CSV)
logs_df  = pd.read_csv(MEALS_CSV)
logs_df["timestamp"] = pd.to_datetime(logs_df["timestamp"])

print(f"Loaded  users: {len(users_df)}   meal rows: {len(logs_df)}")


Users CSV : /home/artemis/project/hobbes_meal_rec/data/test data/ten_users_gpt-4o.csv
Meals CSV : /home/artemis/project/hobbes_meal_rec/data/test data/test_meal_logs.csv
Loaded  users: 10   meal rows: 264


In [5]:
sbert = SentenceTransformer("all-MiniLM-L6-v2")

def make_history_summary(user_id):
    recent = logs_df[logs_df.user_id==user_id].copy()
    recent = recent[recent.timestamp >= datetime.now()-timedelta(days=7)]
    # flavour vector
    emb = sbert.encode(recent["meal_name"].tolist() or [""],
                       normalize_embeddings=True).mean(axis=0)
    # 7‑day macro avg
    daily = recent.groupby(recent.timestamp.dt.date)[["kcal","protein_g","carbs_g","fat_g"]].sum()
    macros = daily.mean().to_dict() if not daily.empty else \
             {"kcal":0,"protein_g":0,"carbs_g":0,"fat_g":0}
    return {"flavor_vec": emb, "macro_7d": macros}

def macro_score_target(plan_df, targets):
    tot = plan_df[["kcal","protein_g","carbs_g","fat_g"]].sum().values
    targ= np.array([targets["optimal_calories"],
                    targets["protein_g"],
                    targets["carbs_g"],
                    targets["fat_g"]])
    rmse = np.sqrt(mean_squared_error(targ, tot))
    return max(0, 1 - rmse / targ[0])

def taste_score(plan_titles, flavor_vec):
    vec = sbert.encode(plan_titles, normalize_embeddings=True)
    return cosine_similarity([vec],[flavor_vec])[0,0]

def day_score(plan_df, flavor_vec, targets,
              beta_macro=0.6, beta_taste=0.4):
    macro = macro_score_target(plan_df, targets)
    taste = taste_score(" ".join(plan_df.title), flavor_vec)
    return beta_macro*macro + beta_taste*taste, macro, taste


/home/artemis/anaconda3/envs/hobbes/lib/python3.10/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
def parse_generated_meals(row):
    json_blob = re.search(r"\{.*\}", row["raw_output"], re.S).group()
    data = json.loads(json_blob)
    meals = []
    for m in data["meals"]:
        meals.append({
            "meal_slot": m["label"],
            "title":     m["name"],
            "kcal":      m["macros"]["calories"],
            "protein_g": m["macros"]["protein_g"],
            "carbs_g":   m["macros"]["carbs_g"],
            "fat_g":     m["macros"]["fat_g"],
        })
    return pd.DataFrame(meals)

gen_plans = {row.user_id: parse_generated_meals(row)
             for _, row in users_df.iterrows()}
print("Parsed GPT‑4o plans for", len(gen_plans), "users.")


Parsed GPT‑4o plans for 10 users.


In [8]:
from datetime import datetime, timedelta
from IPython.display import display, HTML

rows = []
LOOKBACK_DAYS = 7
cutoff        = datetime.now() - timedelta(days=LOOKBACK_DAYS)

for _, u in users_df.iterrows():
    uid = u.user_id
    hist = make_history_summary(uid)
    plan_df = gen_plans[uid]

    targets = {"optimal_calories": u.optimal_calories,
               "protein_g": u.protein_g,
               "carbs_g":   u.carbs_g,
               "fat_g":     u.fat_g}

    score, macro, taste = day_score(plan_df, hist["flavor_vec"], targets)

    rows.append({"user_id": uid,
                 "day_score": round(score,3),
                 "macro": round(macro,3),
                 "taste": round(taste,3)})

    # ---------- display block ----------
    print(f"\n=== User {uid}  •  day_score {score:.3f} "
          f"(macro {macro:.3f} · taste {taste:.3f}) ===")

    # 7‑day history
    recent_df = (logs_df[(logs_df.user_id==uid) & (logs_df.timestamp>=cutoff)]
                 .sort_values("timestamp")
                 .loc[:, ["timestamp","meal_slot","meal_name","kcal","protein_g"]])
    print("👣  Last 7 days")
    display(recent_df)

    # plan under evaluation
    print("📋  GPT‑4o plan")
    display(plan_df[["meal_slot","title","kcal","protein_g"]])

score_df = pd.DataFrame(rows)



=== User 1  •  day_score 0.778 (macro 0.998 · taste 0.447) ===
👣  Last 7 days


,timestamp,meal_slot,meal_name,kcal,protein_g
5,2025-04-28 21:00:00,Lunch,Fish & Chips,1100,40.0
6,2025-04-29 03:00:00,Dinner,Fish & Chips,1100,40.0
7,2025-04-29 16:00:00,Breakfast,Club Sandwich,838,35.0
8,2025-04-29 21:00:00,Lunch,Fish & Chips,1100,40.0
9,2025-04-30 03:00:00,Dinner,Milkshake (250 ml),525,12.5
10,2025-04-30 16:00:00,Breakfast,Fish & Chips,1100,40.0
11,2025-04-30 21:00:00,Lunch,Club Sandwich,838,35.0
12,2025-05-01 03:00:00,Dinner,Fish & Chips,1100,40.0
13,2025-05-01 16:00:00,Breakfast,Milkshake (250 ml),525,12.5
14,2025-05-01 21:00:00,Lunch,Fish & Chips,1100,40.0


📋  GPT‑4o plan


,meal_slot,title,kcal,protein_g
0,Breakfast,Tofu Scramble with Avocado Toast,620,30
1,Lunch,Chickpea Quinoa Salad,860,40
2,Evening Snack,Greek Yogurt with Berries and Nuts,250,20
3,Dinner,Lentil and Vegetable Stir-fry,740,95



=== User 2  •  day_score 0.815 (macro 0.993 · taste 0.548) ===
👣  Last 7 days


...

,meal_slot,title,kcal,protein_g
0,Breakfast,Greek Yogurt with Berries and Almonds,510,30
1,Lunch,Grilled Chicken Salad with Quinoa,715,55
2,Evening Snack,Hummus with Carrot Sticks,200,7
3,Dinner,Baked Salmon with Sweet Potato and Broccoli,617,61



=== User 10  •  day_score 0.788 (macro 0.993 · taste 0.481) ===
👣  Last 7 days


,timestamp,meal_slot,meal_name,kcal,protein_g
241,2025-04-28 21:00:00,Lunch,Fish & Chips,660,24.0
242,2025-04-29 00:00:00,Snack,Club Sandwich,502,21.0
243,2025-04-29 03:00:00,Dinner,Milkshake (250 ml),315,7.5
244,2025-04-29 16:00:00,Breakfast,Taco Plate (3 tacos),600,21.0
245,2025-04-29 21:00:00,Lunch,Milkshake (250 ml),315,7.5
246,2025-04-30 00:00:00,Snack,Fish & Chips,660,24.0
247,2025-04-30 03:00:00,Dinner,Milkshake (250 ml),315,7.5
248,2025-04-30 16:00:00,Breakfast,Fried Rice w/ Egg,562,15.0
249,2025-04-30 21:00:00,Lunch,Taco Plate (3 tacos),600,21.0
250,2025-05-01 00:00:00,Snack,Milkshake (250 ml),315,7.5


📋  GPT‑4o plan


,meal_slot,title,kcal,protein_g
0,Breakfast,Oatmeal with Berries and Almonds,560,18
1,Lunch,Grilled Chicken Salad,780,65
2,Evening Snack,Greek Yogurt with Honey and Walnuts,300,20
3,Dinner,Quinoa and Black Bean Stir-fry,601,65


In [ ]:
print("=== GPT‑4o plan scores ===")
display(score_df)
